In [1]:
import os
import sys
import csv
import datetime
import shutil
import random
import glob
import pandas as pd
import numpy as np
from scipy import stats
import subprocess

In [3]:
clinvar = "/home/galaxy/data/disease/ClinVar/total_result.txt"
eqtl_dir = "/home/galaxy/data/QTL/m6a_related_SNPs/eQTL/m6a_qtl_bed/"
eqtl_list = glob.glob("%s/*.bed" % eqtl_dir)
result_dir = "/home/galaxy/project/QTL_analysis/related_with_disease/ClinVar/eQTL"

In [7]:
names = ["Disease", "ID", "Accession", "Revies status", "Rs ID", "Significant", "Study name"]
df_clinvar = pd.read_table(clinvar, sep="\t", header=None, names=names)
df_clinvar.head()

,Disease,ID,Accession,Revies status,Rs ID,Significant,Study name
0,multiple conditions,36037642,SCV000266334.1,no assertion criteria provided,rs758432471,Pathogenic,UniProtKB:ClinVar:OMIM:HGNC:MedGen:HP:ClinGen
1,Left ventricular noncompaction cardiomyopathy,36540647,SCV000390731.2,"criteria provided, single submitter",rs886051092,Uncertain significance,ClinVar:OMIM:HGNC:MedGen:ClinGen
2,See cases,27389576,SCV000584735.1,no assertion criteria provided,NaN,Microarray,HGNC:dbVar:HP:OMIM
3,Porphobilinogen synthase deficiency,35468001,SCV000476652.2,"criteria provided, single submitter",rs142213018,Likely benign,SNOMED CT:ClinVar:OMIM:HGNC:Orphanet:MedGen:Cl...
4,not specified,36562345,SCV000332741.4,"criteria provided, single submitter",rs201967064,Likely benign,ClinVar:OMIM:HGNC:MedGen:ClinGen


In [11]:
for eqtl in eqtl_list:
    result_file = os.path.join(result_dir, os.path.basename(eqtl).replace(".bed", ".txt"))
    df_eqtl = pd.read_table(eqtl, sep="\t", header=None, names=["Chr", "Start", "End", "Info"])
    df_eqtl["Rs ID"] = df_eqtl.iloc[:, 3].str.split(";").str[2]
    df_eqtl["Gene"] = df_eqtl.iloc[:, 3].str.split(";").str[0]
#     print(df_eqtl.head())
    df_merge = df_eqtl.merge(df_clinvar, how="left")
    df_merge.to_csv(result_file, sep="\t", index=False)

#### parse ClinVar result

In [3]:
raw_dir = "/home/galaxy/project/QTL_analysis/related_with_disease/ClinVar/eQTL"
raw_list = glob.glob("%s/*.txt" % raw_dir)
for raw in raw_list:
    df = pd.read_table(raw, sep="\t")
    print(df.head())
    df_disease = df[~df["Disease"].isna()]
    print(df_disease.head())
    sys.exit(1)

    Chr    Start      End                                               Info  \
0  chr1   843188   843189  ENSG00000188976.6;0.0125;rs2977615;A;G;chr1:84...   
1  chr1   857099   857100  ENSG00000188976.6;0.0125;rs2905036;C;T;chr1:85...   
2  chr1  1300329  1300330  ENSG00000235098.4;0.1625;rs640712;T;A;chr1:130...   
3  chr1  1307602  1307603  ENSG00000235098.4;0.1625;rs2887285;G;A;chr1:13...   
4  chr1  1319062  1319063  ENSG00000235098.4;0.125;rs1886772;G;A;chr1:131...   

       Rs ID               Gene Disease  ID Accession Revies status  \
0  rs2977615  ENSG00000188976.6     NaN NaN       NaN           NaN   
1  rs2905036  ENSG00000188976.6     NaN NaN       NaN           NaN   
2   rs640712  ENSG00000235098.4     NaN NaN       NaN           NaN   
3  rs2887285  ENSG00000235098.4     NaN NaN       NaN           NaN   
4  rs1886772  ENSG00000235098.4     NaN NaN       NaN           NaN   

  Significant Study name  
0         NaN        NaN  
1         NaN        NaN  
2         N

SystemExit: 1

/home/galaxy/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3275: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
